In [3]:
import pandas as pd
import numpy as np

# Explanation of SARIMA parameters

In this notebook we develop documentation for easy reference that tell us what these mighty and enigmatic SARIMA model orders are. That's p, d, q, P, D, Q, S.

SARIMA model is a time series model that lets you predict future values of some variable(s) of interest. It stands for Seasonal AutoRegressive Integrated Moving Average. Really, this this three racoons models standing on eachother's shoulders in a trench coat. It will be helpful to consider each little racoon as its own entity before looking at the whole coated figure.

- Autoregressive (AR) model.
A model unto its own! Tries to predict a future value based on values in the past. For example, what are tomorrow's used car sales? How many days back is most helpful for allowing us to predict tomorrow's sales? Only yesterday? Only 3 days ago? In essence, we're drawing a best fit line through a scatter plot of lagged data (the x values) and the current data (y values) to see which one has the least amount of error.

- Moving Average (MA) model.

# To do list! Open Questions!

- How are the B's determined? Is that the work of the SARIMA model? How does it do that if so? 
- How is e determined?
- How do the equations for p (autoregressive) and q (moving average) integrate to form an ARIMA model?
- How do we determine scedacity?

### What is little p

p is the order of your autoregressive model. It is to how many lags back the model is to observe previous values of the target variable. If you're lookin to wrangle your p paramter pardner, you will be looking at the AR piece of the SARIMA model. Forget the other stuff for now, it's AR time. It'll start making sense when you take a look at the equation for an autoregresesive model 

The basic equation for an autoregressive model is this:

y<sub>t</sub> = B<sub>0</sub> + B<sub>1</sub>y<sub>t-1</sub> + e<sub>t</sub>

y<sub>t</sub> = the value you're trying to predict<br />
y<sub>t-1</sub> = the previous value

B<sub>n</sub> = The coefficient of y at lag n. Consider y<sub>t</sub> = 3 + 2y<sub>t-1</sub> + e<sub>t</sub> <br /> 3 is the B value at t = 0 (B<sub>0</sub> = 3) and 2 is the B value at t-1 (B<sub>1</sub> = 2)

e = Error. All time series equations have inherent error, and e represents it. 

The y's can be subscripted down to whatever lag it needs to be. So like t-1 is one period ago, but it can be all the way back to t-n. 
So if I want to predict tomorrow's sales, that's y<sub>t</sub>. Today's sales would be y<sub>t-1</sub.

So if you have an autoregressive equation that goes back 3 time periods then it's going to have all the way to t-3. E.g. an AR(3) would be 

y<sub>t</sub> = B<sub>0</sub> + B<sub>1</sub>y<sub>t-1</sub> + B<sub>2</sub>y<sub>t-2</sub> + B<sub>3</sub>y<sub>t-3</sub> + e<sub>t</sub>

### Identifying p

The PACF is most useful for identifying the order of an autoregressive model. "large PACF values at a given lag indicate this value as a possible choice for the order of an autoregressive model."

Should check PACF and ACF plots though and look for a pattern as well as when it dips below significance threshold. 

Values lying outside of either of these bounds (the significance thresholds of the PACF plots) are indicative of an autoregressive process.

*At this point we're curious about walking through the mathematical determination of a p. Specifically, we want to look at the relationship between the pacf plot and the p-value that you'll get for linear regression models at different lags.*

In pursuance of the above italicized text:
- grab some time series data (keep it simple)
- generate pacf plot of it
- run linear regression models at various time lags that seem relevant (probably just those hopeful few time lags that are above the significance threshold)
- compare corresponding p-values to their lag on the pacf plot
- Hypothesis: The last significant lag should be the one with the lowest (and therefore best) p_value. And that lag will be your p choice.

# Bibliography 

https://online.stat.psu.edu/stat501/book/export/html/996#:~:text=The%20order%20of%20an%20autoregression,value%20at%20the%20present%20time.